# Simple Code Agent
Perform login to HuggingFace Hub.

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

## Playlist
Search for music recommendation to build a party playlist on DuckDuckGo.

In [ ]:
from smolagents import CodeAgent, DuckDuckGoSearchTool, HfApiModel

agent = CodeAgent(tools=[DuckDuckGoSearchTool()], model=HfApiModel())

agent.run("Search for the best music recommendations for a party at the Wayne's mansion.")

## Menu
Create a menu to prepare.

In [ ]:
from smolagents import CodeAgent, tool, HfApiModel


# define a custom tool that suggests the menu based on the occasion
@tool
def suggest_menu(occasion: str) -> str:
    """Returns a menu suggestion based on the occasion.

    Args:
        occasion: str, the occasion name. Supported values are "casual", "formal",
            and "superhero".

    Returns:
        str, the suggested menu.
    """
    if occasion == "casual":
        return "Pizza, snacks, and drinks."
    elif occasion == "formal":
        return "3-course dinner with wine and dessert."
    elif occasion == "superhero":
        return "Buffet with high-energy and healthy food."
    else:
        return "custom menu for the butler."


# create the agent creating the menu
agent = CodeAgent(tools=[suggest_menu], model=HfApiModel())

# prepare the menu for the party
agent.run("Prepare a formal menu for the party.")

## Prepare everything for the party
Prepare everything for the party.

In [ ]:
from smolagents import CodeAgent, HfApiModel
import numpy as np
import time
import datetime

agent = CodeAgent(tools=[], model=HfApiModel(), additional_authorized_imports=["datetime"])

agent.run(
    """
    Alfred needs to prepare for the party. Here are the tasks:
    1. Prepare the drinks - 30 minutes
    2. Decorate the mansion - 60 minutes
    3. Set up the menu - 45 minutes
    4. Prepare the music and playlist - 45 minutes

    If we start right now, at what time will the party be ready?
    """
)

## Upload agent
Push agent to hub

In [ ]:
agent.push_to_hub("pierclgr/AlfredAgent")

## Download agent
Download agent from HF Hub and execute it.

In [ ]:
alfred_agent = agent.from_hub("pierclgr/AlfredAgent", trust_remote_code=True)
alfred_agent.run(
    "Give me the best playlist for a party at Wayne's mansion. The party idea is a 'villain masquerade' theme")

## Create party planning Agent
Create a complete party planning agent with many tools.

In [ ]:
from smolagents import CodeAgent, DuckDuckGoSearchTool, FinalAnswerTool, HfApiModel, Tool, tool, VisitWebpageTool


@tool
def suggest_menu(occasion: str) -> str:
    """Returns a menu suggestion based on the occasion.

    Args:
        occasion (str): The occasion name. Supported values are "casual", "formal",
            and "superhero".

    Returns:
        str: The suggested menu.
    """
    if occasion == "casual":
        return "Pizza, snacks, and drinks."
    elif occasion == "formal":
        return "3-course dinner with wine and dessert."
    elif occasion == "superhero":
        return "Buffet with high-energy and healthy food."
    else:
        return "Custom menu for the butler."


@tool
def catering_service_tool(query: str) -> str:
    """Returns the highest rated catering service in Gotham City.

    Args:
        query (str): The search query.

    Returns:
        str: The highest rated catering service.
    """
    # example list of catering services and their ratings
    services = {
        "Gotham Catering Co.": 4.9,
        "Wayne Manor Catering": 4.8,
        "Gotham City Events": 4.7
    }

    # find the highest rated catering service (simulating search query filtering)
    best_service = max(services, key=services.get)

    return best_service


class SuperheroPartyThemeTool(Tool):
    """
    A tool that suggests creative superhero-themed party ideas based on a category.

    Attributes:
        name (str): The name of the tool.
        description (str): A brief description of the tool.
        inputs (dict): A dictionary describing the input parameters.
        output_type (str): The type of the output.

    """

    name = "superhero_party_theme_generator"
    description = """
    This tool suggests creative superhero-themed party ideas based on a category.
    It returns a unique party theme idea."""

    inputs = {
        "category": {
            "type": "string",
            "description": "The type of superhero party (e.g., 'classic heroes', 'villain masquerade', 'futuristic Gotham').",
        }
    }

    output_type = "string"

    def forward(self, category: str):
        """
        Generates a superhero-themed party idea based on the given category.

        Args:
            category (str): The type of superhero party.

        Returns:
            str: A unique party theme idea.
        """
        themes = {
            "classic heroes": "Justice League Gala: Guests come dressed as their favorite DC heroes with themed cocktails like 'The Kryptonite Punch'.",
            "villain masquerade": "Gotham Rogues' Ball: A mysterious masquerade where guests dress as classic Batman villains.",
            "futuristic Gotham": "Neo-Gotham Night: A cyberpunk-style party inspired by Batman Beyond, with neon decorations and futuristic gadgets."
        }

        return themes.get(category.lower(),
                          "Themed party idea not found. Try 'classic heroes', 'villain masquerade', or 'futuristic Gotham'.")


agent = CodeAgent(
    tools=[
        DuckDuckGoSearchTool(),
        VisitWebpageTool(),
        suggest_menu,
        catering_service_tool,
        SuperheroPartyThemeTool()
    ],
    model=HfApiModel(),
    max_steps=10,
    verbosity_level=2
)

agent.run(
    "Give me the best playlist for a party at the Wayne's mansion. The party idea is a 'villain masquerade' theme")

## Agent inspection
Inspect the agent

In [ ]:
import os
import base64

LANGFUSE_PUBLIC_KEY = "pk-..."  # insert you langfuse public key
LANGFUSE_SECRET_KEY = "sk-..."  # insert you langfuse secret key
LANGFUSE_AUTH = base64.b64encode(f"{LANGFUSE_PUBLIC_KEY}:{LANGFUSE_SECRET_KEY}".encode()).decode()

os.environ["OTEL_EXPORTER_OTLP_ENDPOINT"] = "https://cloud.langfuse.com/api/public/otel"  # EU data region
# os.environ["OTEL_EXPORTER_OTLP_ENDPOINT"] = "https://us.cloud.langfuse.com/api/public/otel" # US data region
os.environ["OTEL_EXPORTER_OTLP_HEADERS"] = f"Authorization=Basic {LANGFUSE_AUTH}"

os.environ["HF_TOKEN"] = "hf_..."  # insert you hugging face token


Track agent performances

In [ ]:
from opentelemetry.sdk.trace import TracerProvider
from openinference.instrumentation.smolagents import SmolagentsInstrumentor
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk.trace.export import SimpleSpanProcessor

trace_provider = TracerProvider()
trace_provider.add_span_processor(SimpleSpanProcessor(OTLPSpanExporter()))

SmolagentsInstrumentor().instrument(tracer_provider=trace_provider)

Run the agent and perform the tracing of performances

In [ ]:
from smolagents import CodeAgent, HfApiModel

agent = CodeAgent(tools=[], model=HfApiModel())

alfred_agent = agent.from_hub('pierclgr/AlfredAgent', trust_remote_code=True)
alfred_agent.run(
    "Give me the best playlist for a party at Wayne's mansion. The party idea is a 'villain masquerade' theme")